# Projeto Machine Learning - Identificador de fruta

**Integrantes:** Rafael da Fonte Freire, Victor Vazquez e Gabriel Miras Floriano


**Turma:** 2°C

# Introdução

O projeto objetiva a identificação de frutas por meio de imagem. Sabemos que o reconhecimento pode apresentar problemas por razões de formato e cor da fruta, como por exemplo a maçã, que em nosso dataset apresenta 10 tipos diferentes. Para que a identificação seja possível, treinamos o modelo com um dataset que consite em imagens sintéticas de diversos ângulos, chamado "Fruit 360". Esse dataset é composto por imagens organizadas primeiramente em três pastas, uma de treino, que será a base que o programa irá utilizar, constituida por 81 outras pastas, cada uma sendo uma fruta diferente. A segunda pasta é uma base de teste com estrutura semelhante a de treinamento, mas que contém imagens que o programa irá testar a eficiencia. E por fim, a última pasta é de teste multiplo, que é composto por imagens reais em que há um fundo diferente do treino e do teste e que simularia uma situação real.

# Imports

In [10]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import pylab as pl
import random
from sklearn import ensemble
from sklearn.metrics import accuracy_score
import numpy as np
import cv2
from sklearn.linear_model import SGDClassifier
import pickle

# Classes e Funções

Para a leitura e abertura dos dados, nós utilizamos o OpenCV, que é a abreviatura de “Open Source Computer Vision Library” que é uma biblioteca multiplataforma utilizada na área óptica computacional.Para que pudessemos organizar de maneira fácil para o acesso nós usamos a estrutura de classes. Para a análise, diminuímos o tamanho da imagem através do método de pirâmide. Esse método é usado geralmente com imagens de tamanhos constantes, que em nosso caso são todas as imagens 100X100. Como estamos tentando identificar o objeto na imagem, criamos outra imagem com resolução diferente da original. Como não sabíamos qual tamanho usar, pedimos ajuda ao professor que nos definiu uma imagem 25X25. Porém tal método apenas consegue dividir a imagem pela metade, então dividimos uma vez, fazendo com que ficasse 50X50, e depois dividimos mais uma vez, no que resultou em uma imagem 25X25, no tamanho que queríamos.

In [11]:
def imgsd(img):
    if type(None) != type(img):
        bla = cv2.pyrDown(img,dstsize = (50,50))
        bla2 = cv2.pyrDown(bla,dstsize = (25,25))
        return bla2
    return img

class Fruits:
    def __init__(self):
        self.train_images = []
        self.train_target = []
        self.test_images  = []
        self.test_target  = []
    def addfruit(self,fruit_name):
        # O nome das imagens era sempre x_100.jpg ou r_x_100.jpg ou r2_x_100.jpg com x entre 0 e 350.
        n = 0
        m = 0
        for c in range(738): #738 foi escolhido pois o numero maximo de fotos era 738
            
            # Train images
            
            img    = imgsd(cv2.imread('fruits/fruits-360/Training/{0}/{1}_100.jpg'.format(fruit_name,c),1))
            img_r  = imgsd(cv2.imread('fruits/fruits-360/Training/{0}/r_{1}_100.jpg'.format(fruit_name,c),1))
            img_r2 = imgsd(cv2.imread('fruits/fruits-360/Training/{0}/r2_{1}_100.jpg'.format(fruit_name,c),1))
            if type(img) != type(None): # Alguns valores de x eram pulados, portanto isso limpa os "None" da lista
                self.train_images.append(img)
                n+=1
            if type(img_r) != type(None):
                self.train_images.append(img_r)
                n+=1
            if type(img_r2) != type(None):
                self.train_images.append(img_r2)
                n+=1
                
            # Test images
                
            test_img    = imgsd(cv2.imread('fruits/fruits-360/Test/{0}/{1}_100.jpg'.format(fruit_name,c),1))
            test_img_r  = imgsd(cv2.imread('fruits/fruits-360/Test/{0}/r_{1}_100.jpg'.format(fruit_name,c),1))
            test_img_r2 = imgsd(cv2.imread('fruits/fruits-360/Test/{0}/r2_{1}_100.jpg'.format(fruit_name,c),1))
            if type(test_img) != type(None):
                self.test_images.append(test_img)
                m+=1
            if type(test_img_r) != type(None):
                self.test_images.append(test_img_r)
                m+=1
            if type(test_img_r2) != type(None):
                self.test_images.append(test_img_r2)
                m+=1
                
        # Targets
                
        for c in range(n):
            self.train_target.append(fruit_name)
        for c in range(m):
            self.test_target.append(fruit_name)
            

# Definindo variáveis

In [12]:
fruits_to_be_added = ["Apple Braeburn", "Apple Golden 1", "Apple Golden 2", "Apple Golden 3", "Apple Granny Smith", "Apple Red 1", "Apple Red 2", "Apple Red 3", "Apple Red Delicious", "Apple Red Yellow", "Apricot", "Avocado", "Avocado", "Banana", "Banana Red", "Cactus fruit", "Cantaloupe 1", "Cantaloupe 2", "Carambula", "Cherry 1", "Cherry 2", "Cherry Rainier", "Cherry Wax Black", "Cherry Wax Red", "Cherry Wax Yellow", "Clementine", "Cocos", "Dates", "Granadilla", "Grapefruit Pink", "Grapefruit White", "Grape Pink", "Grape White", "Grape White 2", "Guava", "Huckleberry", "Kaki", "Kiwi", "Kumquats", "Lemon", "Lemon Meyer", "Limes", "Lychee", "Mandarine", "Mango", "Maracuja", "Melon Piel de Sapo", "Mulberry", "Nectarine", "Orange", "Papaya", "Passion Fruit", "Peach", "Peach Flat", "Pear", "Pear Abate", "Pear Monster", "Pear Williams", "Pepino", "Physalis", "Physalis with Husk", "Pineapple", "Pineapple Mini", "Pitahaya Red", "Plum", "Pomegrabate", "Quince", "Rambutan", "Raspberry", "Salak", "Strawberry", "Tamarillo", "Tangelo", "Tomato 1", "Tomato 2", "Tomato 3", "Tomato 4", "Tomato Cherry Red", "Tomato Maroon", "Walnut"]

fruits = Fruits()
for c in fruits_to_be_added:
    fruits.addfruit(c)

#treinamento
n_samples      = len(fruits.train_target)
n_samples_test = len(fruits.test_target)

X_train = np.array(fruits.train_images).reshape(n_samples,-1)
y_train = np.array(fruits.train_target)

#teste
X_test = np.array(fruits.test_images).reshape(n_samples_test,-1)
y_test = np.array(fruits.test_target)

#shufflelando
shuffle_index = np.random.permutation(len(X_train))
X_train = X_train[shuffle_index]
y_train = y_train[shuffle_index]

# Treinando e salvando o modelo

** Classificação Binária**

In [13]:
'''
y_train_lemon = (y_train == 'Lemon')
y_test_lemon = (y_test == 'Lemon')
n = 10
for original, binarized in zip(y_train[:n], y_train_lemon[:n]):
    print('{} -> {}'.format(original, binarized))
    
y_train_lemon
'''

"\ny_train_lemon = (y_train == 'Lemon')\ny_test_lemon = (y_test == 'Lemon')\nn = 10\nfor original, binarized in zip(y_train[:n], y_train_lemon[:n]):\n    print('{} -> {}'.format(original, binarized))\n    \ny_train_lemon\n"

**SGDClassifier**

In [14]:
'''
sgd_clf = SGDClassifier(max_iter=50, tol=None)#, random_state=RANDOM_SEED)  
sgd_clf.fit(X_train, y_train)

y_pred_fruits = sgd_clf.predict(X_test)

t = 0
for i in range(len(y_test)):
    if y_test[i] == y_pred_fruits[i]:
        t += 1
print(t/len(y_test))    
        
print() 
#Saving model
pickle.dump(sgd_clf, open('fruit_classifier.sav', 'wb'))
'''

"\nsgd_clf = SGDClassifier(max_iter=50, tol=None)#, random_state=RANDOM_SEED)  \nsgd_clf.fit(X_train, y_train)\n\ny_pred_fruits = sgd_clf.predict(X_test)\n\nt = 0\nfor i in range(len(y_test)):\n    if y_test[i] == y_pred_fruits[i]:\n        t += 1\nprint(t/len(y_test))    \n        \nprint() \n#Saving model\npickle.dump(sgd_clf, open('fruit_classifier.sav', 'wb'))\n"

# Carregando o modelo

**Loading model**

In [15]:
fruit_clf = pickle.load(open('fruit_classifier.sav', 'rb'))

**Testing**

In [16]:
fruit_clf.predict([X_test[12300]])
prob = fruit_clf.score(X_test,y_test)*100
print("A acurácia do nosso modelo é de {0:.2f}%".format(prob))

A acurácia do nosso modelo é de 86.05%


# Modelo em prática

In [17]:
real_test = imgsd(cv2.resize(cv2.imread('test3.jpg',1),(100,100))).reshape(1,-1)

fruit_clf.predict(real_test)

array(['Physalis with Husk'], dtype='<U19')

# Dataset License

MIT License

Copyright (c) 2017-2018 Mihai Oltean, Horea Muresan

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Conclusão

oi